In [1]:
import pandas as pd
import requests
import plotly.express as px
import pickle
token = "pk.eyJ1IjoibG1hZ25hbmEiLCJhIjoiY2s2N3hmNzgwMGNnODNqcGJ1N2l2ZXZpdiJ9.-aOxDLM8KbEQnJfXegtl7A"

ModuleNotFoundError: No module named 'pandas'

Appel à l'API MapBox 

In [ ]:
def request_route(lat1, long1, lat2, long2):
    coord = str(long1)+","+str(lat1)+";"+str(long2)+","+str(lat2)
    return requests.get("https://api.mapbox.com/directions/v5/mapbox/cycling/"+coord, 
                            params={"alternatives": "true", "geometries": "geojson", "steps": "true", "access_token": token})
    

Remplissage du dictionaire :

In [ ]:
def fill_dict(file):
    file_already_used = False
    with open("files_done.txt",'rb') as infile: #check if the file has already been used
        files_done = pickle.load(infile)
        for f in files_done:
            if(f == file):
                file_already_used = True
                break
                
    with open("trips_washington.dict",'rb') as infile:
        dict_trips = pickle.load(infile)
        
    if(not(file_already_used)):
        #Filling without mapbox's request
        print("fucj")
        for index, row in trips.iterrows(): #for all trips 
            break
            key = str(row["Start station number"])+','+str(row["End station number"])#keys are stations' number
            if(key in dict_trips): #store the duration in dict_trips
                dict_trips[key]['durations'].append(row['Duration'])
                dict_trips[key]['number_of_trip']+=1
            else:
                dict_trips[key] = {'durations': [row['Duration']], 'routes': [], 'number_of_trip': 1}


    trips = pd.read_csv("Datas/America/Washington/"+file)
    stations = pd.read_csv("Datas/America/Washington/Capital_Bike_Share_Locations.csv")
            
    #Filling without mapbox's request
    for key in dict_trips: #for all trips in dict_trips
        if(dict_trips[key]['routes'] == []): #if there is no route for the trip
            save_route = True
            tab_stations = key.split(',')
            st1 = stations[stations['TERMINAL_NUMBER']==int(tab_stations[0])] #find the position of the stations
            st2 = stations[stations['TERMINAL_NUMBER']==int(tab_stations[1])]
            if(len(st1)>0 and len(st2)>0): #if the stations' position have been found
                lon1 = st1['LONGITUDE'].values[0]
                lat1 = st1['LATITUDE'].values[0]
                lon2 = st2['LONGITUDE'].values[0]
                lat2 = st2['LATITUDE'].values[0]
                req = request_route(lat1, lon1, lat2, lon2) #mapbox request to find a route between the stations
                response = req.json()
                if(response['code']=='Ok'): #if a route have been found
                    steps = response['routes'][0]['legs'][0]['steps'] #we browse all the steps of the route
                    for step in steps:
                        if(step['maneuver']['instruction'].find("Wharf") != -1):
                            save_route = False #if the route is not good (using a boat) we don't save it
                            break
                    if(save_route): #if we save the route
                        df_temp = pd.DataFrame.from_records(response['routes'][0]['geometry']['coordinates'], 
                                                   columns=['lon', 'lat']) #create a DF from the route (nparray)
                        dict_trips[key]['routes'].append(df_temp) #save the DF in dict_trips
                    else: #if we don't save we store an empty DF in dict_trips
                        dict_trips[key]['routes'].append(pd.DataFrame(columns=['lon', 'lat', 'route_num']))
                    with open('trips_washington.dict', 'wb') as outfile: #save dict_trips
                        pickle.dump(dict_trips, outfile)
                
    with open('trips_washington.dict', 'wb') as outfile: #save dict_trips
        pickle.dump(dict_trips, outfile)
        
    files_done.append(file) #append the used file to the list of used files
    with open('files_done.txt', 'wb') as outfile: #save the used files list
        pickle.dump(files_done, outfile)
                
                
        

In [ ]:
with open('files_done.txt', 'wb') as outfile:
        pickle.dump(["2012Q1-capitalbikeshare-tripdata.csv"], outfile)

fill_dict("2012Q1-capitalbikeshare-tripdata.csv")

In [ ]:
dict_trips = {}
with open('trips_washington.dict', 'rb') as infile: #load a dictionary in dict_trips
    dict_trips = pickle.load(infile)
    
route_num = 0
df = pd.DataFrame(columns=['lon', 'lat', 'route_num']) #creation of an empty dataframe
for key in dict_trips: #for all trips in dict_trips
    if(len(dict_trips[key]['routes'])!=0 and not(dict_trips[key]['routes'][0].empty)): #if a route is found
        df_temp = dict_trips[key]['routes'][0] #we take the route
        df_temp['route_num'] = route_num #we add a number to differentiate routes
        route_num+=1
        df = df.append(df_temp) #we add the route to the final DF
df

In [ ]:
def display(dfdisplay, n=75, line_group="route_num", color=None):
    """
    Display a dataframe of gps points on a mapbox map.
    Parameters
    ----------
    df or str : pandas' DataFrame with columns=['lat', 'lon', 'route_num'] or the name of a file containing one
        Dataframe to display or the file where it is located
    n : int, optional
        Number of routes to display
    line_group : str, optional
        Dataframe's attribute used to differenciate routes
    color : str, optional
        Dataframe's attribute used to color routes
    """
    n+=1
    if(type(dfdisplay) == str): #if df is a file location
        with open(dfdisplay,'rb') as infile:
            dfdisplay = pickle.load(infile) #open the file to load the dataframe
    dfdisplay = dfdisplay[dfdisplay[line_group]<n]
    fig = px.line_mapbox(dfdisplay, lat="lat", lon="lon", line_group=line_group, color=color, zoom=11)
    fig.show()

In [ ]:
display(df)